## Train LSTM model 
Notebook describing general procedure to build the Phynteny model. <br> 
From here work on code to select the optimal number of memory cells, batch size, layers

In [11]:
#imports 
import pickle
import pandas as pd
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Bidirectional, TimeDistributed, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.compat.v1.keras.layers import CuDNNLSTM,CuDNNGRU
import tensorflow as tf
import format_data

ModuleNotFoundError: No module named 'format_data'

In [2]:
#need to read in training genomes 
file = open("/home/grig0076/phispy_phrog_pickles/phrogtrainingdata_phrogs_integrasestartend_fourormore.pkl",'rb')
training_data = pickle.load(file)
file.close()

training_keys = list(training_data.keys())

annot = pd.read_csv('/home/grig0076/LSTMs/phrog_annot_v4.tsv', sep = '\t')
cat_dict = dict(zip([str(i) for i in annot['phrog']], annot['category']))
cat_dict[None] = 'unknown function'

one_letter = {'DNA, RNA and nucleotide metabolism' : 4,
 'connector' : 2,
 'head and packaging' : 3,
 'integration and excision': 1,
 'lysis' : 5,
 'moron, auxiliary metabolic gene and host takeover' : 6,
 'other' : 7,
 'tail' : 8,
 'transcription regulation' : 9,
 'unknown function' :  0 ,}

#use this dictionary to generate an encoding of each phrog
phrog_encoding = dict(zip([str(i) for i in annot['phrog']], [one_letter.get(c) for c in annot['category']]))

#add a None object to this dictionary which is consist with the unknown 
phrog_encoding[None] = one_letter.get('unknown function') 

In [4]:
training_encodings, features = format_data.format_data(training_data) 

num_functions = len(one_letter)
max_length = np.max([len(t) for t in training_encodings])
n_features = num_functions + len(features) 

NameError: name 'format_data' is not defined

In [ ]:
train_num = 100000
test_num = 10000

#generate random indices
idx = [i for i in range(len(training_keys))]
random.shuffle(idx)

X_train, y_train = generate_dataset([training_encodings[i] for i in idx[:train_num]], 
                                    [[f[j] for f in features] for j in idx[:train_num]],
                                    num_functions, 
                                    n_features, 
                                    max_length)

X_test, y_test = generate_dataset([training_encodings[i] for i in idx[train_num: test_num]], 
                                  [[f[j] for f in features] for j in idx[train_num: test_num]],
                                    num_functions, 
                                    n_features, 
                                    max_length)

In [ ]:
#conditions for the early stopping 
patience = 3
min_delta = 0.5

#conditions which can be tuned 
memory_cells = 50 
batch_size = 128 
epochs = 15 
es = EarlyStopping(monitor='loss', mode='min', verbose=2, patience=patience, min_delta=min_delta)

model = Sequential() 
model.add(Bidirectional(CuDNNLSTM(memory_cells, return_sequences=True),input_shape = (max_length, n_features) ))
model.add(Dropout(0.2))
model.add(Bidirectional(CuDNNLSTM(memory_cells, return_sequences = True)))
model.add(Dropout(0.2))
model.add(Bidirectional(CuDNNLSTM(memory_cells, return_sequences = True)))
model.add(TimeDistributed(Dense(num_functions, activation='softmax')))
model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics = ['acc']) 
print(model.summary())

In [ ]:
history = model.fit(X_train, y_train, epochs = epochs, batch_size = batch_size, callbacks = [es, history], validation_split = 0.1)

In [ ]:
#evaluate the model 
correct = 0 
for i in range(1000): 
    X, y = generate_example(training_encodings[i], [f[i] for f in features], num_functions, n_features, max_length) 
    yhat = model.predict(X) 
    
    #the pre decoded sequences from softmax act as probabilities 
    if [one_hot_decode(i) for i in yhat] == [one_hot_decode(i) for i in y]: 
        correct += 1 
print('Accuracy: %f' % ((correct/1000)*100.0))